### imports and libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time

### API info

Insert your api key here from the EIA website (https://www.eia.gov/opendata/).

In [3]:
api_key= '' 

Here is a list and dictionary of query codes that will be used in the API URL to get the appropriate energy,date value wanted for you problem. Can be edited however you wish. 

In [5]:
query_list= ['D', 'NG', 'NG.COL', 'NG.WAT', 'NG.NG', 'NG.NUC', 'NG.OTH', 'NG.OIL', 
              'NG.SUN', 'NG.WND', 'TI', 'DF']
query_dict= {'D':'demand', 'NG':'net_generation', 'NG.COL':'coal', 'NG.WAT':'hydro', 
              'NG.NG':'natural_gas', 'NG.NUC':'nuclear', 'NG.OTH':'other', 
              'NG.OIL':'petroleum', 'NG.SUN':'solar', 'NG.WND':'wind', 
              'TI':'total_interchange', 'DF':'forecast'}

Here is a list of balancing authorities used in the API URL specifically for this project. Can be amended to include more balancing authorities you are interested in.

In [51]:
plant_list= ['ERCO', 'SCEG']

### functions

Function to extract date and energy amount from the API URL and return as a dataframe.

In [7]:
def extract_energy(res): 
    if res.status_code == 200:
        ts= res.json()['series'][0]['data'] # location of information from json file
        df= pd.DataFrame(ts)
    return df

Function to use the above extract_energy function to return a dataframe based upon your api_key, plant_list, and specific query

In [17]:
def make_energy_df(api_key, plant_list, query):
    # for plant in plant_list:
    url= f'http://api.eia.gov/series/?api_key={api_key}&series_id=EBA.{plant}-ALL.{query}.HL'
    res= requests.get(url)

    if 'data' not in (res.json().keys()):       
        energy_df= extract_energy(res)
        return energy_df
        
    else:
        return 'no data'

### make energy dataframe

This for-loop aggregates all the dataframes created using the make_energy_df function over each query in the query_list and each plant in the plant_list. It then sorts the dataframe by plant and then by datetime.

In [52]:
plant_energy= pd.DataFrame()
for plant in plant_list:
    energy= pd.DataFrame()
    for query in query_list:
        if query == 'D':
            energy_df= make_energy_df(api_key, plant_list, query)
            #energy_df['plant']= plant
            energy= energy.append(energy_df, ignore_index= True)
            energy.columns= ['datetime', 'demand']#, 'plant']
            energy.set_index('datetime', inplace= True)
    
        else:
            energy_df= make_energy_df(api_key, plant_list, query)
            if 'no data' not in energy_df:
            
                energy_df.columns= ['datetime', query_dict[query]]
                energy_df.set_index('datetime', inplace= True)
        
                energy= pd.merge(energy, energy_df[query_dict[query]], 
                                 on= 'datetime', how= 'outer')
                energy['plant']= plant

    plant_energy= plant_energy.append(energy)
    
    if len(plant_list) == 1:
        plant_energy.sort_values('datetime', ascending= False, inplace= True)
    else:
        plant_energy.sort_values(['plant', 'datetime'], ascending= [True, False], inplace= True)


Saves dataframe to csv file. Can input your own title after '../data/' to make it more understandble for you. It is commented out to not unintentionally overwrite the csv file used for the analyses when this specific project occured.

In [12]:
# plant_energy.to_csv('../data/all_erco_energy_cst.csv')